In [ ]:
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
files = os.listdir("../tour_grade_time_series")
files = [os.path.join("../tour_grade_time_series", f) for f in files if os.path.isfile(os.path.join("../tour_grade_time_series", f))]


In [ ]:
df = pd.DataFrame()

for file in tqdm(files):

    tmp = pd.read_csv(file)
    df = pd.concat([df, tmp], axis=0)

    print(df.shape)

In [ ]:
df = df.fillna("")
df = df[df["week_day"]!=""]

In [ ]:
df['SeasonFrom'] = pd.to_datetime(df['SeasonFrom'])
df['SeasonTo'] = pd.to_datetime(df['SeasonTo'])

In [ ]:
# Products should be active at least in 2022

cutoff_date = pd.to_datetime('2022-01-01')
df = df[df['SeasonTo'] >= cutoff_date]

# If future dates, keep current date as maximum

cutoff_date = pd.to_datetime('2024-07-01')
df.loc[df['SeasonTo'] > cutoff_date, 'SeasonTo'] = cutoff_date

# If start dates is in the future, keep current date as minimum

cutoff_date = pd.to_datetime('2024-07-01')
df.loc[df['SeasonFrom'] > cutoff_date, 'SeasonFrom'] = cutoff_date

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.to_pickle("../tour_grade_time_series.pickle")

# Preprocess data

In [39]:
import pandas as pd
from tqdm import tqdm

In [40]:
df = pd.read_pickle("../tour_grade_time_series.pickle")

In [41]:
del df["start_time"]

In [42]:
df.fillna("", inplace=True)

In [ ]:
df = df[df["adult_retail_price"]!=""]

In [ ]:
week_day_map = {
    'M': 0,  # Monday
    'T': 1,  # Tuesday
    'W': 2,  # Wednesday
    'H': 3,  # Thursday
    'F': 4,  # Friday
    'S': 5,  # Saturday
    'U': 6   # Sunday
}

def generate_dates(row):
    
    start_date = row['SeasonFrom']
    end_date = row['SeasonTo']
    week_day = week_day_map[row['week_day']]
    
    # Generate all dates between start_date and end_date
    all_dates = pd.date_range(start=start_date, end=end_date)
    
    # Filter dates to keep only those that match the week_day
    filtered_dates = all_dates[all_dates.weekday == week_day]
    
    # Create new rows for each filtered date
    new_rows = pd.DataFrame({
        'ProductCode': row['ProductCode'],
        'TourGradeCode': row['TourGradeCode'],
        'Date': filtered_dates,
        'adult_retail_price': row['adult_retail_price']
    })
    
    return new_rows

In [ ]:
final_df = pd.DataFrame()

for prd in tqdm(list(set(list(df["ProductCode"])))):

    subset = df[df["ProductCode"]==prd]

    for tgc in list(set(list(subset["TourGradeCode"]))):

        subset_now = subset[subset["TourGradeCode"]==tgc]

        # Take mean of prices for changes in the same weekday and period
        subset_mean = subset_now.groupby(['ProductCode', 'TourGradeCode', 'SeasonFrom', 'SeasonTo', 'week_day'], as_index=False)['adult_retail_price'].mean()
        
        expanded_df = pd.concat(subset_mean.apply(generate_dates, axis=1).values, ignore_index=True)
        expanded_df_ordered = expanded_df.sort_values(by='Date', ascending=True)

        date_range = pd.date_range(start=expanded_df_ordered['Date'].min(), end=expanded_df_ordered['Date'].max())
        expanded_df_ordered = expanded_df_ordered.set_index('Date').reindex(date_range).reset_index()

        expanded_df_ordered['adult_retail_price'] = expanded_df_ordered['adult_retail_price'].fillna(method='ffill')
        expanded_df_ordered['adult_retail_price'] = expanded_df_ordered['adult_retail_price'].fillna(method='bfill')

        expanded_df_ordered = expanded_df_ordered.rename(columns={'index': 'Date'})

        expanded_df_ordered.to_pickle(f"../tmp/time_series_{prd}_{tgc}.pickle")

  0%|          | 0/463689 [00:00<?, ?it/s]/tmp/ipykernel_446871/3362201808.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  expanded_df_ordered['adult_retail_price'] = expanded_df_ordered['adult_retail_price'].fillna(method='ffill')
/tmp/ipykernel_446871/3362201808.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  expanded_df_ordered['adult_retail_price'] = expanded_df_ordered['adult_retail_price'].fillna(method='bfill')
  0%|          | 0/463689 [00:02<?, ?it/s]


SystemExit: 

/home/fabio/Desktop/data-exploration/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
